### Diagnostic Information Classification
Outputs:
* Group - 01: Rhythm File
* Group - 02: Duration File
* Group - 03: Amplitude File
* Group - 04: Morphology File
* Group - 05: Unlabeled File

In [1]:
import pandas as pd
import ast
import numpy as np

# Load the Chapman-Ningbo file and retrieve related codes for the specified group
def load_related_codes_by_group(label_file_path, group_name):
    related_codes = set()
    label_data = pd.read_csv(label_file_path, delimiter=";")
    
    # Filter codes for the specified group
    group_data = label_data[label_data['Group'] == group_name]
    
    # Retrieve all Integration Codes for the group
    related_codes.update(group_data['Snomed Code'].astype(str).tolist())
    return related_codes

# Function to map Acronym Name -> Snomed Code
def find_related_acronyms_with_codes(row, acronym_to_code_map):
    try:
        names = ast.literal_eval(row['DiagnosisName']) if isinstance(row['DiagnosisName'], str) else []
        
        # Map Acronym Name and Snomed Code
        related_acronyms = [name for name in names if name in acronym_to_code_map]
        related_codes = [acronym_to_code_map[name] for name in related_acronyms if name in acronym_to_code_map]
        
        return related_codes, related_acronyms
    except (ValueError, SyntaxError):
        return [], []

# Load the existing DiagnosticInfo_all.csv file and create files for each group while adding related codes
def create_group_files_and_add_columns(input_csv, label_file_path):
    # Load the DiagnosticInfo_all.csv file
    all_patient_info = pd.read_csv(input_csv)

    # Adjust the PatientAge column as needed
    all_patient_info['PatientAge'] = all_patient_info['PatientAge'].apply(
        lambda x: np.nan if x in [0.0, 0] or pd.isna(x) else x
    )

    # Define group names and filtering columns
    groups = ['Rhythm', 'Duration', 'Amplitude', 'Morphology', 'Unlabeled']

    # Retrieve Acronym Name -> Snomed Code mappings from the Chapman-Ningbo file
    label_data = pd.read_csv(label_file_path, delimiter=";")
    acronym_to_code_map = label_data.set_index('Acronym Name')['Snomed Code'].to_dict()

    # Filter data and create new CSV files for each group
    for group in groups:
        # Filter rows where the group column is 'True'
        filtered_df = all_patient_info[all_patient_info[group] == True]

        # Select relevant columns
        filtered_df = filtered_df[['ID', 'PatientAge', 'Gender', 'SnomedCode', 'DiagnosisName', 'MergedSnomedCode', 'MergedDiagnosisName']]

        # Retrieve codes related to the specified group
        related_codes = load_related_codes_by_group(label_file_path, group)

        # Function to process new columns
        def find_related_info(row):
            try:
                # Process MergedSnomedCode and MergedDiagnosisName columns
                codes = ast.literal_eval(row['MergedSnomedCode']) if isinstance(row['MergedSnomedCode'], str) else []
                names = ast.literal_eval(row['MergedDiagnosisName']) if isinstance(row['MergedDiagnosisName'], str) else []
                
                # Find related codes and corresponding diagnoses
                related_info = [(code, names[i]) for i, code in enumerate(codes) if code in related_codes]
                
                # Return lists of codes and diagnoses
                related_codes_list = list(set(info[0] for info in related_info)) if related_info else []
                related_names_list = list(set(info[1] for info in related_info)) if related_info else []
                
                return related_codes_list, related_names_list
            except (ValueError, SyntaxError):
                return [], []

        # Populate the new columns
        filtered_df[[f'Merged{group}Code', f'Merged{group}']] = filtered_df.apply(
            lambda row: find_related_info(row),
            axis=1,
            result_type="expand"
        )

        # Find related codes and names for RawRhythmCode and RawRhythm
        def find_raw_info(row):
            try:
                # Check SnomedCode and DiagnosisName columns
                codes = ast.literal_eval(row['SnomedCode']) if isinstance(row['SnomedCode'], str) else []
                names = ast.literal_eval(row['DiagnosisName']) if isinstance(row['DiagnosisName'], str) else []
                
                # Use the smallest length if lengths differ
                min_len = min(len(codes), len(names))

                # Map related codes and corresponding diagnoses
                related_info = [(codes[i], names[i]) for i in range(min_len) if codes[i] in related_codes]
                
                # Return lists of codes and diagnoses
                related_codes_list = list(set(info[0] for info in related_info)) if related_info else []
                related_names_list = list(set(info[1] for info in related_info)) if related_info else []
                
                return related_codes_list, related_names_list
            except (ValueError, SyntaxError):
                return [], []

        # Populate the new columns
        filtered_df[[f'Raw{group}Code', f'Raw{group}']] = filtered_df.apply(
            lambda row: find_raw_info(row),
            axis=1,
            result_type="expand"
        )

        # Define the final column order
        final_columns = [
            'ID', 'PatientAge', 'Gender', 'SnomedCode', 'DiagnosisName', 
            'MergedSnomedCode', 'MergedDiagnosisName', 
            f'Raw{group}Code', f'Raw{group}', 
            f'Merged{group}Code', f'Merged{group}'
        ]

        # Reorder columns
        filtered_df = filtered_df[final_columns]

        # Generate the output file name
        output_file = f"DiagnosticInfo_{group.lower()}.csv"
        
        # Save the CSV file
        filtered_df.to_csv(output_file, index=False)
        print(f"{group} group file has been created and updated: {output_file}")

# Define file paths
input_csv = "DiagnosticInfo_all.csv"  # Path to DiagnosticInfo_all.csv
label_file_path = "../01_SNOMED_CT_Code/Chapman_Ningbo_ECG_DB_Labeling_Info.csv"  # Path to Chapman-Ningbo file

# Run the function
create_group_files_and_add_columns(input_csv, label_file_path)


Rhythm group file has been created and updated: DiagnosticInfo_rhythm.csv
Duration group file has been created and updated: DiagnosticInfo_duration.csv
Amplitude group file has been created and updated: DiagnosticInfo_amplitude.csv
Morphology group file has been created and updated: DiagnosticInfo_morphology.csv
Unlabeled group file has been created and updated: DiagnosticInfo_unlabeled.csv


In [2]:
import pandas as pd

# Try to load the DiagnosticInfo_rhythm.csv file
diagnosis_info_rhythm_path = "DiagnosticInfo_rhythm.csv"

try:
    # Read the DiagnosticInfo_rhythm.csv file into a DataFrame
    diagnostic_info_rhythm_df = pd.read_csv(diagnosis_info_rhythm_path)

except FileNotFoundError:
    print(f"Error: The file {diagnosis_info_rhythm_path} was not found. Please check the path and try again.")

diagnostic_info_rhythm_df.head(5)

,ID,PatientAge,Gender,SnomedCode,DiagnosisName,MergedSnomedCode,MergedDiagnosisName,RawRhythmCode,RawRhythm,MergedRhythmCode,MergedRhythm
0,JS00001,85.0,Male,"['164889003', '59118001', '164934002']","['AF', 'NBBB', 'TWC']","['164889003', '59118001', '55930002']","['AFIB', 'NBBB', 'STTA']",['164889003'],['AF'],['164889003'],['AFIB']
1,JS00002,59.0,Female,"['426177001', '164934002']","['SB', 'TWC']","['426177001', '55930002']","['SB', 'STTA']",['426177001'],['SB'],['426177001'],['SB']
2,JS00004,66.0,Male,['426177001'],['SB'],['426177001'],['SB'],['426177001'],['SB'],['426177001'],['SB']
3,JS00005,73.0,Female,"['164890007', '429622005', '428750005']","['AFL', 'STDD', 'STTC']","['164889003', '55930002', '55930002']","['AFIB', 'STTA', 'STTA']",['164890007'],['AFL'],['164889003'],['AFIB']
4,JS00006,46.0,Female,['426177001'],['SB'],['426177001'],['SB'],['426177001'],['SB'],['426177001'],['SB']


#### Additional Cardiac Conditions (Rhythm)

In [3]:
# Load DiagnosticInfo_all.csv and DiagnosticInfo_rhythm.csv files
def update_rhythm_file_with_conditions(all_info_file, rhythm_file):
    # Load the files
    all_info_df = pd.read_csv(all_info_file)
    diagnostic_info_rhythm_df = pd.read_csv(rhythm_file)
    
    # Match rows using the 'ID' column
    for index, rhythm_row in diagnostic_info_rhythm_df.iterrows():
        patient_id = rhythm_row['ID']
        
        # Find the same ID in DiagnosticInfo_all.csv
        all_row = all_info_df[all_info_df['ID'] == patient_id]
        
        if not all_row.empty:
            # Check the four values other than Rhythm
            duration_true = all_row['Duration'].iloc[0] == True
            amplitude_true = all_row['Amplitude'].iloc[0] == True
            morphology_true = all_row['Morphology'].iloc[0] == True
            unlabeled_true = all_row['Unlabeled'].iloc[0] == True

            # If any of these are TRUE, set AdditionalCardiacConditions to TRUE
            if any([duration_true, amplitude_true, morphology_true, unlabeled_true]):
                diagnostic_info_rhythm_df.at[index, 'AdditionalCardiacConditions'] = True
            else:
                diagnostic_info_rhythm_df.at[index, 'AdditionalCardiacConditions'] = False

    # Save the updated data
    diagnostic_info_rhythm_df.to_csv(rhythm_file, index=False)
    print(f"{rhythm_file} has been updated with AdditionalCardiacConditions.")

# Define file paths
all_info_file = "DiagnosticInfo_all.csv"
rhythm_file = "DiagnosticInfo_rhythm.csv"

# Run the function
update_rhythm_file_with_conditions(all_info_file, rhythm_file)


DiagnosticInfo_rhythm.csv has been updated with AdditionalCardiacConditions.


### Explaratory Data Analysis (Rhythm)

In [4]:
import pandas as pd

# Try to load the DiagnosticInfo_rhythm.csv file
diagnosis_info_rhythm_path = "DiagnosticInfo_rhythm.csv"

try:
    # Read the DiagnosticInfo_rhythm.csv file into a DataFrame
    diagnostic_info_rhythm_df = pd.read_csv(diagnosis_info_rhythm_path)

except FileNotFoundError:
    print(f"Error: The file {diagnosis_info_rhythm_path} was not found. Please check the path and try again.")

diagnostic_info_rhythm_df.head(10)

,ID,PatientAge,Gender,SnomedCode,DiagnosisName,MergedSnomedCode,MergedDiagnosisName,RawRhythmCode,RawRhythm,MergedRhythmCode,MergedRhythm,AdditionalCardiacConditions
0,JS00001,85.0,Male,"['164889003', '59118001', '164934002']","['AF', 'NBBB', 'TWC']","['164889003', '59118001', '55930002']","['AFIB', 'NBBB', 'STTA']",['164889003'],['AF'],['164889003'],['AFIB'],True
1,JS00002,59.0,Female,"['426177001', '164934002']","['SB', 'TWC']","['426177001', '55930002']","['SB', 'STTA']",['426177001'],['SB'],['426177001'],['SB'],True
2,JS00004,66.0,Male,['426177001'],['SB'],['426177001'],['SB'],['426177001'],['SB'],['426177001'],['SB'],False
3,JS00005,73.0,Female,"['164890007', '429622005', '428750005']","['AFL', 'STDD', 'STTC']","['164889003', '55930002', '55930002']","['AFIB', 'STTA', 'STTA']",['164890007'],['AFL'],['164889003'],['AFIB'],True
4,JS00006,46.0,Female,['426177001'],['SB'],['426177001'],['SB'],['426177001'],['SB'],['426177001'],['SB'],False
5,JS00007,80.0,Female,"['164889003', '164934002']","['AF', 'TWC']","['164889003', '55930002']","['AFIB', 'STTA']",['164889003'],['AF'],['164889003'],['AFIB'],True
6,JS00008,46.0,Male,['426783006'],['SR'],['426783006'],['SR'],['426783006'],['SR'],['426783006'],['SR'],False
7,JS00009,45.0,Male,['426177001'],['SB'],['426177001'],['SB'],['426177001'],['SB'],['426177001'],['SB'],False
8,JS00010,47.0,Female,['426177001'],['SB'],['426177001'],['SB'],['426177001'],['SB'],['426177001'],['SB'],False
9,JS00011,63.0,Male,"['426177001', '55827005']","['SB', 'LVH']","['426177001', '55827005']","['SB', 'LVH']",['426177001'],['SB'],['426177001'],['SB'],True


In [5]:
# Shape of dataset
print("Shape of Dataset:", diagnostic_info_rhythm_df.shape)

Shape of Dataset: (43948, 12)


#### Unique Value (Cleaned Multiple Rhythm)

In [6]:
unique_number = []
for i in diagnostic_info_rhythm_df.columns:
    x = diagnostic_info_rhythm_df[i].value_counts().count()
    unique_number.append(x)
    
pd.DataFrame(unique_number, index = diagnostic_info_rhythm_df.columns, columns = ["Total Unique Values"])

,Total Unique Values
ID,43948
PatientAge,86
Gender,2
SnomedCode,4721
DiagnosisName,4652
MergedSnomedCode,3874
MergedDiagnosisName,3874
RawRhythmCode,37
RawRhythm,37
MergedRhythmCode,12


In [7]:
# Load the DataFrame (assuming the file is already loaded into diagnostic_info_df)
diagnostic_info_rhythm_df = pd.read_csv(diagnosis_info_rhythm_path)

# Calculate the distribution of Rhythm values
rhythm_counts = diagnostic_info_rhythm_df['RawRhythm'].value_counts()

# Display the results in DataFrame format
rhythm_counts_df = pd.DataFrame(rhythm_counts).reset_index()
rhythm_counts_df.columns = ['RawRhythm', 'Count']

# Display the final DataFrame
rhythm_counts_df

,RawRhythm,Count
0,['SB'],15807
1,['AFL'],7936
2,['SR'],7729
3,['ST'],7126
4,['AF'],1780
5,['SI'],1619
6,"['SI', 'SB']",700
7,['SVT'],623
8,"['SR', 'SI']",201
9,['AT'],141


In [8]:
# Load the DataFrame (assuming the file is already loaded into diagnostic_info_df)
diagnostic_info_rhythm_df = pd.read_csv(diagnosis_info_rhythm_path)

# Calculate the distribution of Rhythm values
rhythm_counts = diagnostic_info_rhythm_df['MergedRhythm'].value_counts()

# Display the results in DataFrame format
rhythm_counts_df = pd.DataFrame(rhythm_counts).reset_index()
rhythm_counts_df.columns = ['MergedRhythm', 'Count']

# Display the final DataFrame
rhythm_counts_df

,MergedRhythm,Count
0,['SB'],15807
1,['AFIB'],9716
2,['SR'],9549
3,['GSVT'],8048
4,"['SR', 'SB']",706
5,"['GSVT', 'AFIB']",53
6,"['SR', 'GSVT']",22
7,"['GSVT', 'SB']",20
8,"['SR', 'AFIB']",18
9,"['SR', 'GSVT', 'SB']",6


In [9]:
# Load the DiagnosticInfo_rhythm.csv file
diagnostic_info_rhythm_path = "DiagnosticInfo_rhythm.csv"
diagnostic_info_rhythm_df = pd.read_csv(diagnostic_info_rhythm_path)

# Remove rows in the RawRhythm column that contain multiple values
diagnostic_info_rhythm_df = diagnostic_info_rhythm_df[diagnostic_info_rhythm_df['RawRhythm'].apply(lambda x: len(eval(x)) == 1)]

# Convert the list in the MergedRhythm column to a single value
diagnostic_info_rhythm_df['MergedRhythm'] = diagnostic_info_rhythm_df['MergedRhythm'].apply(lambda x: eval(x)[0])

# Convert the list in the RawRhythm column to a single value
diagnostic_info_rhythm_df['RawRhythm'] = diagnostic_info_rhythm_df['RawRhythm'].apply(lambda x: eval(x)[0])

# Save the updated file
diagnostic_info_rhythm_df.to_csv(diagnostic_info_rhythm_path, index=False)

print("Rows with multiple values in MergedRhythm and RawRhythm columns have been removed, and the file has been updated.")

Rows with multiple values in MergedRhythm and RawRhythm columns have been removed, and the file has been updated.


In [10]:
# Shape of dataset
print("Shape of Dataset:", diagnostic_info_rhythm_df.shape)

Shape of Dataset: (42802, 12)


In [11]:
unique_number = []
for i in diagnostic_info_rhythm_df.columns:
    x = diagnostic_info_rhythm_df[i].value_counts().count()
    unique_number.append(x)
    
pd.DataFrame(unique_number, index = diagnostic_info_rhythm_df.columns, columns = ["Total Unique Values"])

,Total Unique Values
ID,42802
PatientAge,86
Gender,2
SnomedCode,4389
DiagnosisName,4320
MergedSnomedCode,3560
MergedDiagnosisName,3560
RawRhythmCode,11
RawRhythm,11
MergedRhythmCode,4


In [12]:
diagnostic_info_rhythm_df.head()

,ID,PatientAge,Gender,SnomedCode,DiagnosisName,MergedSnomedCode,MergedDiagnosisName,RawRhythmCode,RawRhythm,MergedRhythmCode,MergedRhythm,AdditionalCardiacConditions
0,JS00001,85.0,Male,"['164889003', '59118001', '164934002']","['AF', 'NBBB', 'TWC']","['164889003', '59118001', '55930002']","['AFIB', 'NBBB', 'STTA']",['164889003'],AF,['164889003'],AFIB,True
1,JS00002,59.0,Female,"['426177001', '164934002']","['SB', 'TWC']","['426177001', '55930002']","['SB', 'STTA']",['426177001'],SB,['426177001'],SB,True
2,JS00004,66.0,Male,['426177001'],['SB'],['426177001'],['SB'],['426177001'],SB,['426177001'],SB,False
3,JS00005,73.0,Female,"['164890007', '429622005', '428750005']","['AFL', 'STDD', 'STTC']","['164889003', '55930002', '55930002']","['AFIB', 'STTA', 'STTA']",['164890007'],AFL,['164889003'],AFIB,True
4,JS00006,46.0,Female,['426177001'],['SB'],['426177001'],['SB'],['426177001'],SB,['426177001'],SB,False


#### Missing Value Check

In [13]:
# Number of missing values
isnull_number = []
for i in diagnostic_info_rhythm_df.columns:
    x = diagnostic_info_rhythm_df[i].isnull().sum()
    isnull_number.append(x)
    
pd.DataFrame(isnull_number, index = diagnostic_info_rhythm_df.columns, columns = ["Total Missing Values"])

,Total Missing Values
ID,0
PatientAge,0
Gender,0
SnomedCode,0
DiagnosisName,0
MergedSnomedCode,0
MergedDiagnosisName,0
RawRhythmCode,0
RawRhythm,0
MergedRhythmCode,0


#### Value Count

In [14]:
# Load the DataFrame (assuming the file is already loaded into diagnostic_info_df)
diagnostic_info_rhythm_df = pd.read_csv(diagnosis_info_rhythm_path)

# Calculate the distribution of Rhythm values
rhythm_counts = diagnostic_info_rhythm_df['RawRhythm'].value_counts()

# Display the results in DataFrame format
rhythm_counts_df = pd.DataFrame(rhythm_counts).reset_index()
rhythm_counts_df.columns = ['RawRhythm', 'Count']

# Display the final DataFrame
rhythm_counts_df

,RawRhythm,Count
0,SB,15807
1,AFL,7936
2,SR,7729
3,ST,7126
4,AF,1780
5,SI,1619
6,SVT,623
7,AT,141
8,AVRT,23
9,AVNRT,16


In [15]:
# Load the DataFrame (assuming the file is already loaded into diagnostic_info_df)
diagnostic_info_rhythm_df = pd.read_csv(diagnosis_info_rhythm_path)

# Calculate the distribution of Rhythm values
rhythm_counts = diagnostic_info_rhythm_df['MergedRhythm'].value_counts()

# Display the results in DataFrame format
rhythm_counts_df = pd.DataFrame(rhythm_counts).reset_index()
rhythm_counts_df.columns = ['MergedRhythm', 'Count']

# Display the final DataFrame
rhythm_counts_df

,MergedRhythm,Count
0,SB,15807
1,AFIB,9716
2,SR,9348
3,GSVT,7931


In [16]:
# Load the DataFrame (assuming the file is already loaded into diagnostic_info_df)
diagnostic_info_rhythm_df = pd.read_csv(diagnosis_info_rhythm_path)

# Calculate the distribution of Rhythm values
additionalcardiac_counts = diagnostic_info_rhythm_df['AdditionalCardiacConditions'].value_counts()

# Display the results in DataFrame format
additionalcardiac_counts_df = pd.DataFrame(additionalcardiac_counts).reset_index()
additionalcardiac_counts_df.columns = ['AdditionalCardiacConditions', 'Count']

# Display the final DataFrame
additionalcardiac_counts_df

,AdditionalCardiacConditions,Count
0,False,21499
1,True,21303


#### Cleaned Data - Target Define

In [17]:
# Load the DiagnosticInfo_rhythm.csv file
diagnostic_info_rhythm_path = "DiagnosticInfo_rhythm.csv"
diagnostic_info_rhythm_df = pd.read_csv(diagnostic_info_rhythm_path)

# Get the unique values in the MergedRhythm column
unique_values = diagnostic_info_rhythm_df['MergedRhythm'].dropna().unique()

# Sort the unique values (alphabetically or as desired)
unique_values = sorted(unique_values)

# Create a column for each class and populate it with 0/1 values
for value in unique_values:
    column_name = f"Target_{value}"  # Example: Target_AFIB, Target_SR
    diagnostic_info_rhythm_df[column_name] = diagnostic_info_rhythm_df['MergedRhythm'].apply(
        lambda x: 1 if x == value else 0
    )

# Save the updated DataFrame
diagnostic_info_rhythm_df.to_csv(diagnostic_info_rhythm_path, index=False)

print(f"Separate 'Target' columns have been added and the CSV file has been updated: {diagnostic_info_rhythm_path}")

Separate 'Target' columns have been added and the CSV file has been updated: DiagnosticInfo_rhythm.csv


In [18]:
diagnostic_info_rhythm_df.head()

,ID,PatientAge,Gender,SnomedCode,DiagnosisName,MergedSnomedCode,MergedDiagnosisName,RawRhythmCode,RawRhythm,MergedRhythmCode,MergedRhythm,AdditionalCardiacConditions,Target_AFIB,Target_GSVT,Target_SB,Target_SR
0,JS00001,85.0,Male,"['164889003', '59118001', '164934002']","['AF', 'NBBB', 'TWC']","['164889003', '59118001', '55930002']","['AFIB', 'NBBB', 'STTA']",['164889003'],AF,['164889003'],AFIB,True,1,0,0,0
1,JS00002,59.0,Female,"['426177001', '164934002']","['SB', 'TWC']","['426177001', '55930002']","['SB', 'STTA']",['426177001'],SB,['426177001'],SB,True,0,0,1,0
2,JS00004,66.0,Male,['426177001'],['SB'],['426177001'],['SB'],['426177001'],SB,['426177001'],SB,False,0,0,1,0
3,JS00005,73.0,Female,"['164890007', '429622005', '428750005']","['AFL', 'STDD', 'STTC']","['164889003', '55930002', '55930002']","['AFIB', 'STTA', 'STTA']",['164890007'],AFL,['164889003'],AFIB,True,1,0,0,0
4,JS00006,46.0,Female,['426177001'],['SB'],['426177001'],['SB'],['426177001'],SB,['426177001'],SB,False,0,0,1,0


### Created Without Additional Cardiac Conditions File

In [19]:
import pandas as pd

# File paths
input_file_path = "DiagnosticInfo_rhythm.csv"
output_file_path = "DiagnosticInfo_rhythm_WithoutAddCarCon.csv"

# Load CSV file
df = pd.read_csv(input_file_path)

# Filter rows with AdditionalCardiacConditions variable False
filtered_df = df[df["AdditionalCardiacConditions"] == False]

# Save filtered data to a new file
filtered_df.to_csv(output_file_path, index=False)

# Process completed

print(f"Without Additional Cardiac Conditions file has been created: {output_file_path}")

Without Additional Cardiac Conditions file has been created: DiagnosticInfo_rhythm_WithoutAddCarCon.csv
